# Scraping Costco Warehouse Addresses

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
from time import sleep

In [2]:
costco_url = "https://www.costco.com/WarehouseListByStateDisplayView"

In [11]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(costco_url)

In [15]:
state_links = [link for link in driver.find_elements(By.CLASS_NAME, "warehouse-title-link") if link.text != ""]

In [16]:
warehouse_links = []

for state_link in state_links:
    state_link.click()
    print(f"{state_link.text:20}", end=" ")
    sleep(1)

    soup = BeautifulSoup(driver.page_source)
    links = [link for link in soup.find_all("a") if link.text == "Get Directions"]
    print(len(links))
    warehouse_links += links

Alabama              4
Louisiana            3
Oklahoma             4
Alaska               6
Maine                1
Oregon               13
Arizona              20
Maryland             11
Pennsylvania         11
Arkansas             1
Massachusetts        6
Puerto Rico          5
California           138
Michigan             17
Colorado             16
Minnesota            13
South Carolina       6
Connecticut          8
Mississippi          1
South Dakota         1
Delaware             1
Missouri             9
Tennessee            8
District of Columbia 1
Montana              5
Texas                39
Florida              33
Nebraska             3
Utah                 14
Georgia              17
Nevada               8
Vermont              1
Hawaii               7
New Hampshire        1
Virginia             17
Idaho                7
New Jersey           21
Washington           33
Illinois             23
New Mexico           3
Indiana              9
New York             19
Wisconsin       

In [22]:
locations = []

for warehouse_link in warehouse_links:
    name = warehouse_link.parent.parent.a.text
    address_lines = warehouse_link.parent.parent.find("div", attrs={"class": "h6-style-guide"}).contents[::2]
    location_data = [name, address_lines[0].strip()] + address_lines[1].strip().replace(",", "").split("\xa0")
    location_data.append(warehouse_link["onclick"][30:-9].replace("%20", " "))

    locations.append(location_data)

In [26]:
if len(locations) == 611:
    costco_locations = pd.DataFrame(locations, columns=["name", "address", "city", "state", "zipcode", "google_query"])
    costco_locations.to_csv("./Data/costco_locations.csv")
    print("Saved!")
else:
    print("Wrong number of warehouses...")

Saved!
